# Agents that can code


In [ ]:
from dotenv import load_dotenv
import autogen

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#llm configuration
llm_config = {
    "model":"gpt-4o-mini",
    "api_key":api_key
}

In [ ]:
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

venv_dir = ".env_llm"
venv_context = create_virtual_env(venv_dir)

executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,
    work_dir="coding",
)
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

## Code executor agent

1. A **code_writer_agent**: this agent is engineer, it will rely on chatGPT to generate python code (other languages are currently not supported by autogen) to accomplish the requested task.
This agent is not a `ConverseableAgent`, it is one of its subcategories, the `AssistantAgent`. The `AssistantAgent` is designed to write code.

In [ ]:
from autogen import AssistantAgent

# Agent that writes code
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_writer_agent_system_message = code_writer_agent.system_message
print(code_writer_agent_system_message)

In [ ]:
from autogen import ConversableAgent

# Code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

## Coding task


In [ ]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. "\
"Create a plot showing the normalized price of NVDA and BTC-USD for the last 4 years "\
"with their 50 weeks moving average. "\
"Make sure the code is in a markdown code block, save the figure"\
" to a file asset_analysis.png and show it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message
)

The code will now execute.
